-sandbox
<img width="280" src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>                   

<br>
> # Bronze para Silver

<br>
## 02 - Investidores 2 
- A pontuação de risco de um investidor indica o quanto ele é um tomador de risco com base em dados históricos. Da mesma forma, a pontuação de risco do ativo é atribuída a um ticker que mostra o quão arriscada ou volátil uma ação é, em uma escala de zero a dez.
- Todos os investidores também têm uma lista de ações favoritas. Esta lista é um produto de dados comportamentais e transacionais.

-sandbox
<h2 style="color:red">Informações:</h2><p>

* Utilizar o `explode`
* Dividir strings com `split`
* Unir DataFrames
* Selecionar colunas após uma junção 

Combine os dados de ações e investidores usando as informações acima mencionadas. 
É necessário para o aplicativo de marketing upstream projetado para direcionar os investidores.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/08-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Cleaning up the learning environment... Removed working directory: dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/02_investidores_2_psp

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 assetsPath : dbfs:/mnt/training/finance-org/solutions/bronze/assets 
 dbfs:/mnt/training/finance-org/solutions/bronze/assets 
 
 investorsPath : dbfs:/mnt/training/finance-org/solutions/silver/investors 
 dbfs:/mnt/training/finance-org/solutions/silver/investors 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/02_investidores_2_psp/silver/investor_favorites 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/02_investidores_2_psp/silver/investor_favorites 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_02_investidores_2_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

Declared the following function:
 realityCheck

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas: 

<br>
- Ler os dados de investidores a partir de **`investmentsPath`** em um DataFrame.
- Manter apenas os seguintes campos: **`investor_id`**, **` name`**, **`credit_score`**, **` risk_score`** e **`fav_stocks`**
- Atualize o DataFrame usando a coluna **`fav_stocks`** para que haja uma única ação para cada linha para um investidor.
- Nomeie o campo explodido para **`asset_id`** e elimine o campo original **`fav_stocks`**.
- Ler os dados de ativos a partir de **`assetsPath`** em um DataFrame.
- Juntar os DataFrames de **`investidores`** e **`ativos`** usando uma junção para a esquerda (**`left`**).
- Grave o DataFrame resultante em **`targetDirectory`**.
- Retorne o DataFrame.

O esquema do DataFrame deve ser:

|name|type|
|---|---|
|investor_id|LongType|
|name|StringType|
|credit_score|DoubleType|
|risk_score|LongType|
|asset_id|StringType|
|asset_risk|IntegerType|

In [0]:
def func_investidores(spark, investorsPath, assetsPath, targetDirectory):
  
  
  from pyspark.sql.functions import col, split, explode
  
  
  # Ler os dados de "investidores" a partir de "investmentsPath" em um DataFrame.
  # Mantendo apenas os campos: "investor_id", "name", "credit_score", "risk_score" e "fav_stocks".
  # Atualiza o DataFrame usando a coluna "fav_stocks", para que haja uma única ação para cada linha para um determinado investidor.
  # Converter em uma lista e depois usando o explode. 
  # Nomeando 0 campo explodido "asset_id" e delete o campo original "fav_stocks".
  investorsDF = (spark.read
                      .format("delta")
                      .load(investorsPath)
                      .select("investor_id", 
                              "name", 
                              "credit_score", 
                              "risk_score", 
                              "fav_stocks")
                      .withColumn('fav_stocks', split(col('fav_stocks'), ","))
                      .select("*", 
                              explode("fav_stocks").alias("asset_id"))
                      .drop("fav_stocks"))
  
  
  # Ler os dados de ativos a partir de "assetsPath" em um DataFrame.
  assets = (spark.read
                 .format("delta")
                 .load(assetsPath))
  
  
  # Juntar os DataFrames de "investidores" e "ativos" usando uma junção para a esquerda (left).
  joinDF = (investorsDF.join(assets, 
                             on = "asset_id", 
                             how = "left")
                       .select(investorsDF["*"], 
                               "asset_risk"))
  
  # Grave o DataFrame resultante em "targetDirectory".
  joinDF.write \
        .mode("overwrite") \
        .format("delta") \
        .option("overwriteSchema", "true") \
        .save(targetDirectory)

  
  # Returna o DataFrame.
  return joinDF

In [0]:
finalDF = func_investidores(spark, investorsPath, assetsPath, targetDirectory)

In [0]:
display(finalDF)

investor_id,name,credit_score,risk_score,asset_id,asset_risk
0,Beverly Hardy,1.5610122744141655,2,AKAM,8
0,Beverly Hardy,1.5610122744141655,2,ANDV,1
0,Beverly Hardy,1.5610122744141655,2,BDX,7
0,Beverly Hardy,1.5610122744141655,2,CHRW,7
0,Beverly Hardy,1.5610122744141655,2,DLTR,0
0,Beverly Hardy,1.5610122744141655,2,EXC,2
0,Beverly Hardy,1.5610122744141655,2,VRSK,5
1,Shirley Goldberg,1.5610122744141655,5,EXR,3
1,Shirley Goldberg,1.5610122744141655,5,F,4
1,Shirley Goldberg,1.5610122744141655,5,KMB,4


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_investidores, spark, investorsPath, assetsPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Silver table in place,
1,Silver table has correct content,


## <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Próximo <p>

> * [Pedidos de empréstimo]($./03-Pedidos emprestimo)

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>